In [1]:
!pwd

/content


In [2]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content"

In [3]:
!kaggle datasets download -d surajghuwalewala/ham1000-segmentation-and-classification

100% 2.58G/2.59G [00:25<00:00, 204MB/s]
100% 2.59G/2.59G [00:25<00:00, 111MB/s]


In [ ]:
!unzip /content/ham1000-segmentation-and-classification.zip

In [5]:
import numpy as np 
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense, Activation,Dropout,Conv2D, MaxPooling2D,BatchNormalization, Flatten
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, load_model, Sequential
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

In [6]:
df_labels = pd.read_csv('../content/GroundTruth.csv')
df_labels['image'] = df_labels['image'] +'.jpg'

labels=['MEL', 'NV', 'BCC', 'AKIEC', 'BKL', 'DF', 'VASC']
label_list=[]
for i in range (len(df_labels)):
    row= list(df_labels.iloc[i])
    del row[0]
    index=np.argmax(row)
    label=labels[index]
    label_list.append(label)
df_labels['label']= label_list
df_labels=df_labels.drop(labels, axis=1)
df_labels['label'].value_counts()

NV       6705
MEL      1113
BKL      1099
BCC       514
AKIEC     327
VASC      142
DF        115
Name: label, dtype: int64

In [7]:
from sklearn.model_selection import train_test_split

tain_size=0.7
valid_size = 0.1
test_size = 0.2
valid_test_split = valid_size / test_size

train_df, valid_test_df = train_test_split(df_labels, train_size=tain_size, shuffle=True, random_state=42)
valid_df, test_df = train_test_split(valid_test_df, train_size=valid_test_split, shuffle=True, random_state=42)

print(' train_df length: ', len(train_df), '  test_df length: ', len(test_df), '  valid_df length: ', len(valid_df))
print (train_df.head())
print (train_df['label'].value_counts())

 train_df length:  7010   test_df length:  1503   valid_df length:  1502
                 image label
2399  ISIC_0026705.jpg    NV
8246  ISIC_0032552.jpg   MEL
8052  ISIC_0032358.jpg   BKL
5840  ISIC_0030146.jpg    NV
3540  ISIC_0027846.jpg   BCC
NV       4686
BKL       784
MEL       758
BCC       362
AKIEC     235
VASC      100
DF         85
Name: label, dtype: int64


In [13]:
sdir=r'../content/images' # main directory where data is stored
save_dir=r'./' # output directory where model will be saved
subject='cancer' # part of the name of the saved model
height=64  # image height
width=64   # image width
channels=3  # number of coloor channels
batch_size=20  # model batch size for training and evaluation
img_shape=(height, width, channels)
img_size=(height, width)

def scalar(img): # unused for efficientnet model
    return img/127.5-1  # scale pixel between -1 and +1
gen=ImageDataGenerator() # no scaling of pixels is need for efficientnet

train_gen=gen.flow_from_dataframe( train_df, sdir, x_col='image', y_col='label', target_size=img_size, class_mode='categorical',
                                    color_mode='rgb', shuffle=True, batch_size=batch_size)

Found 7010 validated image filenames belonging to 7 classes.


In [15]:
valid_gen=gen.flow_from_dataframe(valid_df, sdir, x_col='image', y_col='label', target_size=img_size, class_mode='categorical',
                                    color_mode='rgb', shuffle=True, batch_size=batch_size)

Found 1502 validated image filenames belonging to 7 classes.


In [18]:
linear_model = tf.keras.Sequential([    
   keras.layers.Flatten(input_shape=(height, width, channels)),                                                                 
   keras.layers.Dense(7)                                 
])
linear_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
linear_model.fit(train_gen, epochs=5,  validation_data=valid_gen)

Epoch 1/5
351/351 [==============================] - 90s 255ms/step - loss: 4.0899 - accuracy: 0.1116 - val_loss: 3.8992 - val_accuracy: 0.1105


KeyboardInterrupt: ignored

In [17]:
linear_model = tf.keras.Sequential([    
   keras.layers.Flatten(input_shape=(height, width, channels)),                                                                 
   keras.layers.Dense(7, activation = 'sigmoid')                                 
])
linear_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
linear_model.fit(train_gen, epochs=batch_size,  validation_data=valid_gen)

Epoch 1/20
351/351 [==============================] - 96s 273ms/step - loss: 69.6839 - accuracy: 0.5903 - val_loss: 47.5323 - val_accuracy: 0.5539
Epoch 2/20
351/351 [==============================] - 95s 270ms/step - loss: 47.3393 - accuracy: 0.5896 - val_loss: 45.4955 - val_accuracy: 0.4687
Epoch 3/20
351/351 [==============================] - 89s 253ms/step - loss: 43.4110 - accuracy: 0.6124 - val_loss: 77.8048 - val_accuracy: 0.3802
Epoch 4/20
351/351 [==============================] - 94s 268ms/step - loss: 44.1109 - accuracy: 0.6124 - val_loss: 49.5286 - val_accuracy: 0.6718
Epoch 5/20
351/351 [==============================] - 91s 261ms/step - loss: 42.2969 - accuracy: 0.6148 - val_loss: 49.2782 - val_accuracy: 0.4667
Epoch 6/20
351/351 [==============================] - 95s 270ms/step - loss: 45.1252 - accuracy: 0.6023 - val_loss: 40.8663 - val_accuracy: 0.6698
Epoch 7/20
351/351 [==============================] - 90s 256ms/step - loss: 46.0044 - accuracy: 0.6208 - val_loss: 35